# Homework 10
## Ziyan Hu

In [1]:
import numpy as np
import math

## Q1
### 1.1

In [5]:
# define the function
def f(x):
    return -np.exp(-x) * np.sin(x)
# implement the bisection method
def bisect(f, a, b, tol):
    fa = f(a)
    fb = f(b)
    if fa * fb > 0:
        print("No root found on interval")
        return None
    n = int(np.ceil(np.log2((b - a) / tol)))
    for i in range(n):
        c = (a + b) / 2
        fc = f(c)
        if fa * fc < 0:
            b = c
            fb = fc
        else:
            a = c
            fa = fc
    return (a + b)/2

In [4]:
# test on interval [0,1.5]
a = 0
b = 1.5
tol = 1e-10

xmin = bisect(f, a, b, tol)
print(f"Minimum occurs at x = {xmin:.8f}")

Minimum occurs at x = 1.50000000


### 1.2

In [6]:
# implement the golden section search
def golden(f, a, b, tol):
    golden_ratio = (math.sqrt(5) + 1) / 2
    c = b - (b - a) / golden_ratio
    d = a + (b - a) / golden_ratio
    fc = f(c)
    fd = f(d)
    while abs(c - d) > tol:
        if fc < fd:
            b = d
            d = c
            fd = fc
            c = b - (b - a) / golden_ratio
            fc = f(c)
        else:
            a = c
            c = d
            fc = fd
            d = a + (b - a) / golden_ratio
            fd = f(d)
    return (b + a) / 2

In [7]:
# test on interval [0,1.5]
a = 0
b = 1.5
tol = 1e-10

xmin = golden(f, a, b, tol)
print(f"Minimum occurs at x = {xmin:.8f}")

Minimum occurs at x = 0.78539817


## Q2
### 2.1

In [8]:
# generate the data
n = 10000
beta = np.array([3, -7, 2, -4, 1, 3, 8, -6, -1, 4])

np.random.seed(42)
z = np.random.normal(size=(n, beta.shape[0] - 1))
designmat = np.column_stack((np.array([1]*z.shape[0]), z))

pvec = np.exp(designmat @ beta) / (1 + np.exp(designmat @ beta))
y = 1 * (np.random.uniform(size=n) < pvec)

# sigmoid function
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# negative log likelihood loss
def NLL(beta, X, y):
    p = sigmoid(X @ beta)
    return -np.mean((y*np.log(p)) + ((1-y)*np.log(1-p)))

# gradient of NLL
def grad(beta, X, y):
    p = sigmoid(X @ beta)
    return X.T @ (p - y) / len(y)

In [11]:
# run gradient descent
betas = [np.zeros(10)]
lr_values = [0.001, 0.002, 0.003, 0.004, 0.005]
for lr in lr_values:
    print(f"Learning rate: {lr}")
    beta = np.zeros(10)
    for i in range(1000):
        beta = beta - lr*grad(beta, designmat, y)
        betas.append(beta)
    print(beta)

Learning rate: 0.001
[ 0.08007082 -0.17447287  0.04223791 -0.09964986  0.02495252  0.07238329
  0.19159068 -0.15010739 -0.02576574  0.100258  ]
Learning rate: 0.002
[ 0.14233889 -0.31268688  0.07657699 -0.17813558  0.04434332  0.12986045
  0.34439024 -0.26820064 -0.04630693  0.17978053]
Learning rate: 0.003
[ 0.1926357  -0.42621827  0.10544735 -0.24228269  0.06005639  0.17712179
  0.47059991 -0.36467407 -0.0632689   0.24505224]
Learning rate: 0.004
[ 0.23471044 -0.522572    0.13045008 -0.2964858   0.07327456  0.21724905
  0.57818474 -0.44621149 -0.07771213  0.30034115]
Learning rate: 0.005
[ 0.27089892 -0.6064548   0.15259768 -0.34349189  0.08471688  0.25219464
  0.67216843 -0.51697567 -0.09030323  0.34836099]


## 2.2

In [9]:
# hessian of NLL
def hessian(beta, X, y):
    p = sigmoid(X @ beta)
    W = np.diag(p * (1 - p))
    return (1/len(y)) * (X.T @ W @ X)

# newton method iterates newton steps til convergence
def newton(beta, X, y, tol=1e-8):
    for i in range(1000):
        beta = beta - np.linalg.inv(hessian(beta,X,y)) @ grad(beta, X, y)
        if np.linalg.norm(grad(beta, X, y)) < tol:
            break
    return beta

In [10]:
# find coefficients
beta0 = np.zeros(10)
beta_newton = newton(beta0, designmat, y, tol=1e-10)
print(beta_newton)

[ 2.92514733 -7.03574735  1.96212515 -3.89908532  0.94898396  2.99687945
  7.85276471 -6.01988302 -1.01650936  3.97249099]
